In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz', spark_conf={'spark.driver.memory': '9G', 'spark.executor.memory': '9G'})

Running on Apache Spark version 2.4.3
SparkUI available at http://p0618.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/analysis/hail-20211015-0831-0.2.62-84fa81b9ea3d.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

## Preparation for main analysis

The matrix table below has already been filtered according to the following criteria:

In [4]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sportsmen-with-corrected-samples.mt')

In [5]:
mt.count()

(9824949, 154)

In [6]:
proper_controls = ['WGS_37b', 'WGS_37c', 'WGS_163b', 'WGS_163d', 'WGS_7120', 'WGS_7142', 'WGS_7143', 'WGS_7152',
 'WGS_7153', 'WGS_85b', 'WGS_147c', 'WGS_180b', 'WGS_185c', 'WGS_6819', 'S_7213', 'S_7227', 'S_7241', 'S_7246', 'S_7254', 'S_7274',
                   'S_7307', '494', '462', '468', '479','492', '490'] 

#7119 jest chory

In [7]:
mt = mt.filter_cols((mt.s.contains('B')) | hl.literal(proper_controls).contains(mt.s))

In [8]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-unrelated-corrected.mt')

2021-10-08 13:30:58 Hail: INFO: wrote matrix table with 9824949 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-unrelated-corrected.mt
    Total size: 6.84 GiB
    * Rows/entries: 6.84 GiB
    * Columns: 568.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  13167 rows (7.01 MiB)


In [46]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-unrelated-corrected.mt')

In [47]:
mt.count()

(9824949, 128)

In [11]:
mt_subset = mt.sample_rows(0.001)

In [12]:
mt_subset.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sportsmen-with-corrected-samples-subset.mt')

2021-10-08 13:36:59 Hail: INFO: wrote matrix table with 9812 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sportsmen-with-corrected-samples-subset.mt
    Total size: 11.85 MiB
    * Rows/entries: 11.85 MiB
    * Columns: 581.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  12 rows (16.06 KiB)


In [13]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sportsmen-with-corrected-samples-subset.mt')

In [14]:
pc_rel = hl.pc_relate(mt_subset.GT, 0.001, k=2, statistics='kin')

2021-10-08 13:37:43 Hail: INFO: hwe_normalized_pca: running PCA using 8139 variants.
2021-10-08 13:38:19 Hail: INFO: pca: running PCA with 2 components...
2021-10-08 16:13:51 Hail: INFO: Wrote all 3 blocks of 9812 x 128 matrix with block size 4096.


In [33]:
pairs = pc_rel.filter(pc_rel['kin'] > 0.2) #there is still some kinship but this should dissappear after joint genotypins is repeated

In [34]:
pairs.show()

2021-10-08 17:34:35 Hail: INFO: wrote matrix with 3 rows and 9812 columns as 3 blocks of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-Uk4BnrFI19iCLkTHh8UZIo.bm
2021-10-08 17:34:35 Hail: INFO: wrote matrix with 9812 rows and 128 columns as 3 blocks of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-OGBad5KbjvPOndMCk4tvbW.bm
2021-10-08 17:34:36 Hail: INFO: wrote matrix with 128 rows and 128 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-TxwfTW7QYUTXgrIfkn7SGS.bm
2021-10-08 17:34:36 Hail: INFO: wrote matrix with 128 rows and 128 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-HjOG9LroHHVlMxDUp6cH0U.bm
2021-10-08 17:34:36 Hail: INFO: wrote matrix with 128 rows and 128 columns as 1 block of size 4096 to /net/scratch/people/plggosborcz/pcrelate-write-read-PUx9Pofy4f3jn2iYat8PUP.bm
2021-10-08 17:34:36 Hail: INFO: Ordering unsorted dataset with network shuffle


,,
i,j,
s,s,kin
str,str,float64


In [17]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt_subset.GT)

mt_subset = mt_subset.annotate_cols(group = hl.cond(mt_subset.s.contains("B"), "sport", "gts"))
mt_subset = mt_subset.annotate_cols(scores = pcs[mt_subset.s].scores)

2021-10-08 16:14:31 Hail: INFO: hwe_normalized_pca: running PCA using 8139 variants.
2021-10-08 16:15:05 Hail: INFO: pca: running PCA with 10 components...
/net/software/local/python/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) cond. (Replaced by hl.if_else) -- Deprecated since version 0.2.59.
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
mt_subset.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset-with-pca.mt')

2021-10-08 17:23:45 Hail: INFO: wrote matrix table with 9812 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset-with-pca.mt
    Total size: 10.03 MiB
    * Rows/entries: 10.02 MiB
    * Columns: 10.86 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  12 rows (12.71 KiB)


In [19]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset-with-pca.mt')

In [23]:
p = hl.plot.scatter(mt_subset.scores[0],
                    mt_subset.scores[1],
                    label=mt_subset.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

In [25]:
pcs = mt_subset.scores[0].collect()

In [26]:
np.std(pcs)*6 #no sample is that far from the center

0.6912298271391693

### additional quality control of chosen variants and more PCA:

In [53]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-unrelated-corrected.mt')

In [54]:
mt = hl.variant_qc(mt)

In [55]:
mt = mt.filter_rows((mt.variant_qc.dp_stats.mean > 10) & (mt.variant_qc.gq_stats.mean > 60))

In [56]:
mt.count()

(8461161, 128)

In [15]:
#mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/joint-dp10-gq60.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp10-gq60.mt')

In [16]:
mt.count()

(8461161, 128)

In [17]:
mt = mt.annotate_rows(n_below_dp_3 = hl.agg.count_where(mt.DP < 3),
                     n_below_gq_30 = hl.agg.count_where(mt.GQ <30))

In [18]:
mt = mt.filter_rows((mt.n_below_dp_3 == 0) & (mt.n_below_gq_30 < 6) & (mt.variant_qc.dp_stats.mean > 15) & (mt.variant_qc.gq_stats.mean > 65))

In [19]:
mt.count()

(7587998, 128)

In [20]:
mt = mt.filter_rows(hl.is_snp(mt.alleles[0], mt.alleles[1]))

In [21]:
mt.count()

(6813337, 128)

In [22]:
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp10-gq60-max5below.mt')
#mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp10-gq60-max5below.mt')


2021-10-13 18:05:45 Hail: INFO: wrote matrix table with 6813337 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp10-gq60-max5below.mt
    Total size: 6.73 GiB
    * Rows/entries: 6.73 GiB
    * Columns: 581.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  6710 rows (6.87 MiB)


### get and remove phased genotypes (technical artifacts)

In [23]:
largemt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint.mt')

In [24]:
largemt.count()

(10751439, 287)

In [25]:
largemt = largemt.annotate_rows(n_phased = hl.agg.count_where(largemt.GT.phased))

In [ ]:
largemt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-phased.mt')

In [30]:
largemt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-phased.mt')

In [5]:
#largemt.filter_rows(largemt.n_phased == 0).checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-phased-filtered.mt')

### Add filter from the large mt:

In [23]:
mt.count()

(6813337, 128)

In [24]:
largemt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-phased-filtered.mt')
mt = mt.filter_rows(hl.is_defined(largemt.rows()[mt.row_key]))

In [25]:
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp10-gq60filtered.mt')

2021-10-13 18:10:01 Hail: INFO: wrote matrix table with 5606700 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp10-gq60filtered.mt
    Total size: 5.36 GiB
    * Rows/entries: 5.36 GiB
    * Columns: 581.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4560 rows (5.56 MiB)


In [156]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp10-gq60filtered.mt')

##  now exclude genotypes in patches:

In [157]:
patches = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/patches-hg38.csv')

2021-10-13 19:27:45 Hail: INFO: Reading table without type imputation
  Loading field 'location' as type str (not specified)
  Loading field 'patch' as type str (not specified)


In [158]:
patches = patches.location.collect()

In [159]:
patches.remove('Chr3:91247723-91553319')

In [160]:
mt = hl.filter_intervals(mt, [hl.parse_locus_interval(x, reference_genome='GRCh38') for x in patches], keep = False)

In [163]:
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-filtered-no-patches.mt')

2021-10-13 19:31:02 Hail: INFO: wrote matrix table with 5540443 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-filtered-no-patches.mt
    Total size: 5.29 GiB
    * Rows/entries: 5.29 GiB
    * Columns: 581.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4560 rows (5.56 MiB)


##  exclude sites with weird AF patterns

In [15]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-filtered-no-patches.mt')

In [16]:
mt = mt.annotate_cols(group = hl.cond(mt.s.contains("B"), "sport", "control"))
pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/sporstmen-pheno.csv', delimiter=';')
pheno = pheno.key_by(pheno.sample_id)
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

/net/software/local/python/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) cond. (Replaced by hl.if_else) -- Deprecated since version 0.2.59.
  """Entry point for launching an IPython kernel.
2021-10-14 10:06:05 Hail: INFO: Reading table without type imputation
  Loading field '\ufeffsport' as type str (not specified)
  Loading field 'type' as type str (not specified)
  Loading field 'age' as type str (not specified)
  Loading field 'sample_id' as type str (not specified)


In [17]:
mt = mt.annotate_rows(controls_non_ref = hl.agg.filter((mt.group == 'control'), hl.agg.count_where(mt.GT.is_non_ref())),
                      sports_non_ref = hl.agg.filter((mt.group == 'sport'), hl.agg.count_where(mt.GT.is_non_ref())),
                      controls_ref = hl.agg.filter((mt.group == 'control'), hl.agg.count_where(mt.GT.is_hom_ref())),
                      sports_ref = hl.agg.filter((mt.group == 'sport'), hl.agg.count_where(mt.GT.is_hom_ref())),
                      speed_non_ref = hl.agg.filter((mt.phenotypes.type == 'speed'), hl.agg.count_where(mt.GT.is_non_ref())),
                      endurance_non_ref = hl.agg.filter((mt.phenotypes.type == 'endurance'), hl.agg.count_where(mt.GT.is_non_ref())),
                      speed_ref = hl.agg.filter((mt.phenotypes.type == 'speed'), hl.agg.count_where(mt.GT.is_hom_ref())),
                      endurance_ref = hl.agg.filter((mt.phenotypes.type == 'endurance'), hl.agg.count_where(mt.GT.is_hom_ref())))

In [18]:
mt = mt.annotate_rows(af_controls = mt.controls_non_ref/(mt.controls_non_ref+mt.controls_ref),
                          af_sport = mt.sports_non_ref/(mt.sports_non_ref+mt.sports_ref))

In [26]:
#mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/filtered-anno.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/filtered-anno.mt')

In [27]:
mt = mt.annotate_rows(flag = hl.if_else(((mt.af_controls > 0.1) | (mt.af_sport > 0.1)), hl.if_else(
    hl.is_defined(mt.gnomad_v3.v3_nfe.AF), hl.if_else(
        ((hl.abs(mt.af_controls - mt.gnomad_v3.v3_nfe.AF) > 0.3) | (hl.abs(mt.af_sport - mt.gnomad_v3.v3_nfe.AF) > 0.3)), 'not', 'ok'), 'not'), 'ok'))

In [28]:
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/flagged.mt')

2021-10-14 10:13:13 Hail: INFO: wrote matrix table with 5540443 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/flagged.mt
    Total size: 5.34 GiB
    * Rows/entries: 5.34 GiB
    * Columns: 1.31 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4560 rows (5.61 MiB)


### filter out flagged variants:

In [31]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/flagged.mt')

In [32]:
mt = mt.filter_rows(mt.flag == 'ok')

In [33]:
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/flagged-filtered.mt')

2021-10-14 10:15:11 Hail: INFO: wrote matrix table with 5282269 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/flagged-filtered.mt
    Total size: 5.01 GiB
    * Rows/entries: 5.01 GiB
    * Columns: 1.31 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4401 rows (5.35 MiB)


### Perform PCA and remove outliers

In [34]:
mt_subset = mt.sample_rows(0.001)

In [35]:
mt_subset.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered.mt')

2021-10-14 10:17:02 Hail: INFO: wrote matrix table with 5339 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered.mt
    Total size: 7.15 MiB
    * Rows/entries: 7.15 MiB
    * Columns: 1.31 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  9 rows (12.79 KiB)


In [36]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt_subset.GT)

mt_subset = mt_subset.annotate_cols(scores = pcs[mt_subset.s].scores)

2021-10-14 10:17:42 Hail: INFO: hwe_normalized_pca: running PCA using 4393 variants.
2021-10-14 10:18:21 Hail: INFO: pca: running PCA with 10 components...


In [37]:
mt_subset.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered_PCA.mt')
#mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered_PCA.mt')

2021-10-14 10:45:27 Hail: INFO: wrote matrix table with 5339 rows and 128 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered_PCA.mt
    Total size: 7.16 MiB
    * Rows/entries: 7.15 MiB
    * Columns: 11.62 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  9 rows (12.79 KiB)


In [45]:
p = hl.plot.scatter(mt_subset.scores[0],
                    mt_subset.scores[1],
                    label=mt_subset.s,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

In [43]:
pcs = mt_subset.scores[1].collect()

In [44]:
np.std(pcs)*6 #no sample is that far from the center

0.6124640074529579

In [71]:
mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered.mt')

In [46]:
to_delete = ['WGS_6819', 'WGS_185c']

In [47]:
mt_subset = mt_subset.filter_cols(hl.literal(to_delete).contains(mt_subset.s), keep = False)

In [48]:
mt_subset.count()

(5339, 126)

In [49]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt_subset.GT)
mt_subset = mt_subset.annotate_cols(scores = pcs[mt_subset.s].scores)

2021-10-14 10:54:33 Hail: INFO: hwe_normalized_pca: running PCA using 4358 variants.
2021-10-14 10:55:12 Hail: INFO: pca: running PCA with 10 components...


In [50]:
mt_subset.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered_PCA_less_samples.mt')
#mt_subset = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered_PCA_less_samples.mt')

2021-10-14 11:24:33 Hail: INFO: wrote matrix table with 5339 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/subset_filtered_PCA_less_samples.mt
    Total size: 7.09 MiB
    * Rows/entries: 7.08 MiB
    * Columns: 11.45 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  9 rows (12.69 KiB)


In [54]:
pcs = mt_subset.scores[1].collect()

In [55]:
np.std(pcs)*6 #no sample is that far from the center

0.6101053014016272

In [57]:
p = hl.plot.scatter(mt_subset.scores[0],
                    mt_subset.scores[1],
                    label=mt_subset.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

In [59]:
mt_subset.count()

(5339, 126)

In [64]:
types = mt_subset.phenotypes.type.collect()

In [67]:
types.count('speed')

50

# re-filter the matrix table to remove PCA outliers:

In [4]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/flagged-filtered.mt')
to_delete = ['WGS_6819', 'WGS_185c']
mt = mt.filter_cols(hl.literal(to_delete).contains(mt.s), keep = False)

In [5]:
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp15-gq65-max5below.mt')

2021-10-14 11:59:38 Hail: INFO: wrote matrix table with 5282269 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp15-gq65-max5below.mt
    Total size: 4.94 GiB
    * Rows/entries: 4.94 GiB
    * Columns: 1.30 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4401 rows (5.30 MiB)


In [6]:
mt.count()

(5282269, 126)

# annotate with genes and hpo

In [16]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/joint-dp15-gq65-max5below.mt')

In [17]:
mt.count()

(5282269, 126)

In [18]:
genes = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))
hpo = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/hpo.tsv', impute = True, no_header=True)

start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

2021-10-14 12:07:22 Hail: INFO: Reading table to impute column types
2021-10-14 12:07:23 Hail: INFO: Finished type imputation
  Loading field 'f0' as type str (imputed)
  Loading field 'f1' as type str (imputed)


In [10]:
genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])

genes = genes.key_by(genes.interval)

In [11]:
mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))

In [12]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-genes-pheno.mt')

2021-10-14 11:59:42 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-14 11:59:43 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-14 11:59:46 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-14 11:59:46 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-14 11:59:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-14 11:59:48 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-14 12:04:26 Hail: INFO: wrote matrix table with 5282269 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-genes-pheno.mt
    Total size: 3.92 GiB
    * Rows/entries: 3.92 GiB
    * Columns: 1.07 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4687 rows (5.80 MiB)


In [19]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-genes-pheno.mt')

In [20]:
mt.count()

(5282269, 126)

In [21]:
mt = mt.filter_rows(mt.within_gene == hl.empty_array(hl.tstr), keep = False)

In [22]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic.mt')

2021-10-14 12:10:31 Hail: INFO: wrote matrix table with 3426184 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic.mt
    Total size: 2.66 GiB
    * Rows/entries: 2.66 GiB
    * Columns: 1.07 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4687 rows (5.80 MiB)


In [23]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic.mt')

In [24]:
go_genes = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/human-genes-with-GO-and-symbols')
all_genes = go_genes['UniProtKB Gene Name symbol'].collect()

len(all_genes)

2021-10-14 12:10:32 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)


23804

In [25]:
mt = mt.filter_rows(hl.any(lambda x: hl.literal(all_genes).contains(x), mt.within_gene))

In [26]:
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-go.mt')

2021-10-14 12:15:29 Hail: INFO: wrote matrix table with 2526938 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-go.mt
    Total size: 2.61 GiB
    * Rows/entries: 2.61 GiB
    * Columns: 1.30 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4687 rows (6.99 MiB)


In [27]:
mt.count()

(2526938, 126)

## lets further filter variants - higher quality qenotypes only

In [101]:
#rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker_all.ht')

#contig_len = hl.contig_length(rpmk.interval.end.contig, reference_genome='GRCh38')
#end_pos = rpmk.interval.end.position
#start_pos = rpmk.interval.start.position
#contig = rpmk.interval.start.contig

#rpmk = rpmk.annotate(larger_int_start = hl.if_else(start_pos == 1,
#                                                   hl.locus(contig, start_pos, reference_genome='GRCh38'),
#                                                   hl.locus(contig, start_pos-1, reference_genome='GRCh38')),
#                     larger_int_end = hl.if_else(end_pos == contig_len,
#                                                 hl.locus(contig, end_pos, reference_genome='GRCh38'),
#                                                 hl.locus(contig, end_pos+1, reference_genome='GRCh38')),
#                     larger_int_start_2 = hl.if_else((start_pos == 1) | (start_pos == 2),
#                                                   hl.locus(contig, start_pos, reference_genome='GRCh38'),
#                                                   hl.locus(contig, start_pos-2, reference_genome='GRCh38')),
#                     larger_int_end_2 = hl.if_else((end_pos == contig_len) | (end_pos == (contig_len-1)),
#                                                 hl.locus(contig, end_pos, reference_genome='GRCh38'),
#                                                 hl.locus(contig, end_pos+2, reference_genome='GRCh38')))

In [102]:
#rpmk.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/repeatmasker_anno.ht')

In [29]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-go.mt')

In [30]:
#for some reason a loop doesn't work here
rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_start'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered0.mt')
    
rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_end'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered1.mt')

rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_start_2'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered2.mt')

2021-10-14 12:16:41 Hail: INFO: Coerced sorted dataset
2021-10-14 12:18:34 Hail: INFO: wrote matrix table with 2526938 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered0.mt
    Total size: 2.61 GiB
    * Rows/entries: 2.61 GiB
    * Columns: 1.30 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4687 rows (6.99 MiB)
2021-10-14 12:18:37 Hail: INFO: Coerced sorted dataset
2021-10-14 12:18:40 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-14 12:22:30 Hail: INFO: wrote matrix table with 2520318 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered1.mt
    Total size: 2.61 GiB
    * Rows/entries: 2.61 GiB
    * Columns: 1.30 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4677 rows (6.98 MiB)
2

In [31]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered2.mt')

In [32]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_end_2'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered.mt')

2021-10-14 12:27:08 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-14 12:30:33 Hail: INFO: wrote matrix table with 2507532 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered.mt
    Total size: 2.59 GiB
    * Rows/entries: 2.59 GiB
    * Columns: 1.30 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4658 rows (6.96 MiB)


## filter for gene lists

In [33]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered.mt')

In [34]:
mt.count()

(2507532, 126)

In [4]:
gene_lists = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/gene-lists/sportsmen_gene_lists_october_2021.csv', quote="\"")

2021-10-15 08:35:40 Hail: INFO: Reading table without type imputation
  Loading field 'brain' as type str (not specified)
  Loading field 'sekletal_muscle' as type str (not specified)
  Loading field 'heart' as type str (not specified)
  Loading field 'Hp_0011804' as type str (not specified)
  Loading field 'Hp_0011805' as type str (not specified)
  Loading field 'endurance_markers_2021' as type str (not specified)
  Loading field 'power_markers_2021' as type str (not specified)
  Loading field 'strength_markers_2021' as type str (not specified)
  Loading field 'soft_tissue_injury_2021' as type str (not specified)


In [5]:
brain_genes = gene_lists.brain.collect()
heart_genes = gene_lists.heart.collect()
skeletal_muscle_genes = gene_lists.sekletal_muscle.collect()
Hp_0011804 = gene_lists.Hp_0011804.collect()
Hp_0011805 = gene_lists.Hp_0011805.collect()
endurance_markers_2021 = gene_lists.endurance_markers_2021.collect()
power_markers_2021 = gene_lists.power_markers_2021.collect()
strength_markers_2021 = gene_lists.strength_markers_2021.collect()
soft_tissue_injury_2021 = gene_lists.soft_tissue_injury_2021.collect()

In [38]:
mt_brain_genes = mt.filter_rows(hl.any(lambda x: hl.literal(brain_genes).contains(x), mt.within_gene))
mt_brain_genes.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/brain-genes.mt')

mt_heart_genes = mt.filter_rows(hl.any(lambda x: hl.literal(heart_genes).contains(x), mt.within_gene))
mt_heart_genes.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/heart-genes.mt')

mt_skeletal_genes = mt.filter_rows(hl.any(lambda x: hl.literal(skeletal_muscle_genes).contains(x), mt.within_gene))
mt_skeletal_genes.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/muscle-genes.mt')

mt_Hp_0011804 = mt.filter_rows(hl.any(lambda x: hl.literal(Hp_0011804).contains(x), mt.within_gene))
mt_Hp_0011804.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/hpo-4.mt')

mt_Hp_0011805 = mt.filter_rows(hl.any(lambda x: hl.literal(Hp_0011805).contains(x), mt.within_gene))
mt_Hp_0011805.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/hpo-5.mt')

mt_endurance_markers_2021 = mt.filter_rows(hl.any(lambda x: hl.literal(endurance_markers_2021).contains(x), mt.within_gene))
mt_endurance_markers_2021.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/endurance.mt')

mt_power_markers_2021 = mt.filter_rows(hl.any(lambda x: hl.literal(power_markers_2021).contains(x), mt.within_gene))
mt_power_markers_2021.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/power.mt')

mt_strength_markers_2021 = mt.filter_rows(hl.any(lambda x: hl.literal(strength_markers_2021).contains(x), mt.within_gene))
mt_strength_markers_2021.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/stren.mt')

mt_soft_tissue_injury_2021 = mt.filter_rows(hl.any(lambda x: hl.literal(soft_tissue_injury_2021).contains(x), mt.within_gene))
mt_soft_tissue_injury_2021.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/soft.mt')

2021-10-14 12:31:44 Hail: INFO: wrote matrix table with 127404 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/brain-genes.mt
    Total size: 133.94 MiB
    * Rows/entries: 133.94 MiB
    * Columns: 1.30 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3188 rows (3.05 MiB)
2021-10-14 12:32:47 Hail: INFO: wrote matrix table with 88946 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/heart-genes.mt
    Total size: 94.29 MiB
    * Rows/entries: 94.29 MiB
    * Columns: 1.30 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3718 rows (3.92 MiB)
2021-10-14 12:33:53 Hail: INFO: wrote matrix table with 141716 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/muscle-genes.mt
    Total size: 156.69 MiB
   

## single variant analysis:

In [4]:
mts = ['sporstmen-intragenic-filtered.mt', 'brain-genes.mt', 'heart-genes.mt', 'muscle-genes.mt', 'hpo-4.mt', 'hpo-5.mt', 'endurance.mt',
'power.mt', 'stren.mt', 'soft.mt']

In [41]:
for mt_name in mts:
    mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/'+mt_name)
    to_test = mt.rows()
    to_test = to_test.select(to_test.controls_non_ref, to_test.sports_non_ref, to_test.controls_ref, to_test.sports_ref, to_test.speed_non_ref, to_test.endurance_non_ref, to_test.speed_ref, to_test.endurance_ref)
    to_test.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-'+mt_name)

2021-10-14 12:44:33 Hail: INFO: wrote table with 2507532 rows in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-sporstmen-intragenic-filtered.mt
    Total size: 42.72 MiB
    * Rows: 42.72 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  4931 rows (85.58 KiB)
2021-10-14 12:45:11 Hail: INFO: wrote table with 127404 rows in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-brain-genes.mt
    Total size: 2.29 MiB
    * Rows: 2.29 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  3188 rows (55.44 KiB)
2021-10-14 12:45:50 Hail: INFO: wrote table with 88946 rows in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-heart-genes.mt
    Total size: 1.62 MiB
    * Rows: 1.62 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Large

In [59]:
for mt_name in mts:
    to_test = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-'+mt_name)
    print(mt_name)
    print(to_test.count())
    print(0.05/to_test.count())
    to_test = to_test.annotate(fisher_vs_controls = hl.fisher_exact_test(hl.int32(to_test.controls_non_ref), 
                                                         hl.int32(to_test.controls_ref),
                                                         hl.int32(to_test.sports_non_ref),
                                                         hl.int32(to_test.sports_ref)),
                              fisher_e_vs_s = hl.fisher_exact_test(hl.int32(to_test.endurance_non_ref), 
                                                         hl.int32(to_test.endurance_ref),
                                                         hl.int32(to_test.speed_non_ref),
                                                         hl.int32(to_test.speed_ref)))
    to_test.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-fisher'+mt_name)
    to_test = to_test.filter((to_test.fisher_vs_controls.p_value < 0.05/to_test.count()) | (to_test.fisher_e_vs_s.p_value < 0.05/to_test.count()))
    print(to_test.count())
                            

sporstmen-intragenic-filtered.mt
2507532
1.9939924993978143e-08


2021-10-14 15:58:17 Hail: INFO: wrote table with 2507532 rows in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-fishersporstmen-intragenic-filtered.mt
    Total size: 83.59 MiB
    * Rows: 83.59 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  4931 rows (183.76 KiB)


7
brain-genes.mt
127404
3.9245235628394717e-07


2021-10-14 16:02:57 Hail: INFO: wrote table with 127404 rows in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-fisherbrain-genes.mt
    Total size: 4.44 MiB
    * Rows: 4.44 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  3188 rows (115.16 KiB)


0
heart-genes.mt
88946
5.621388258044207e-07


...

0
stren.mt
10458
4.7810288774144194e-06


2021-10-14 16:11:32 Hail: INFO: wrote table with 10458 rows in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-fisherstren.mt
    Total size: 423.28 KiB
    * Rows: 423.27 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  2294 rows (75.08 KiB)


0
soft.mt
2660
1.879699248120301e-05


2021-10-14 16:12:20 Hail: INFO: wrote table with 2660 rows in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-fishersoft.mt
    Total size: 160.73 KiB
    * Rows: 160.72 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  1004 rows (34.66 KiB)


0


In [67]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/sporstmen-intragenic-filtered.mt')

In [61]:
#annotate with results of each test:

mtables = []
mts = ['sporstmen-intragenic-filtered.mt', 'brain-genes.mt', 'heart-genes.mt', 'muscle-genes.mt', 'hpo-4.mt', 'hpo-5.mt', 'endurance.mt',
'power.mt', 'stren.mt', 'soft.mt']

for mtable in mts:
    mtables.append(hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-test-fisher'+mtable))
    print(mtable)
    
mt = mt.annotate_rows(fisher_all_vs_controls= mtables[0][mt.row_key]['fisher_vs_controls'],
                fisher_all_e_vs_s= mtables[0][mt.row_key]['fisher_e_vs_s'],
                fisher_brain_vs_controls=mtables[1][mt.row_key]['fisher_vs_controls'],
                fisher_brain_e_vs_s=mtables[1][mt.row_key]['fisher_e_vs_s'],
                fisher_heart_vs_controls=mtables[2][mt.row_key]['fisher_vs_controls'],
                fisher_heart_e_vs_s=mtables[2][mt.row_key]['fisher_e_vs_s'],
                fisher_muscle_vs_controls=mtables[3][mt.row_key]['fisher_vs_controls'],
                fisher_muscle_e_vs_s=mtables[3][mt.row_key]['fisher_e_vs_s'],
                fisher_hpo4_vs_controls=mtables[4][mt.row_key]['fisher_vs_controls'],
                fisher_hpo4_e_vs_s=mtables[4][mt.row_key]['fisher_e_vs_s'],
                fisher_hpo5_vs_controls=mtables[5][mt.row_key]['fisher_vs_controls'],
                fisher_hpo5_e_vs_s=mtables[5][mt.row_key]['fisher_e_vs_s'],
                fisher_end_vs_controls=mtables[6][mt.row_key]['fisher_vs_controls'],
                fisher_end_e_vs_s=mtables[6][mt.row_key]['fisher_e_vs_s'],
                fisher_power_vs_controls=mtables[7][mt.row_key]['fisher_vs_controls'],
                fisher_power_e_vs_s=mtables[7][mt.row_key]['fisher_e_vs_s'],
                fisher_stren_vs_controls=mtables[8][mt.row_key]['fisher_vs_controls'],
                fisher_stren_e_vs_s=mtables[8][mt.row_key]['fisher_e_vs_s'],
                fisher_soft_vs_controls=mtables[9][mt.row_key]['fisher_vs_controls'],
                fisher_soft_e_vs_s=mtables[9][mt.row_key]['fisher_e_vs_s']
                )

sporstmen-intragenic-filtered.mt
brain-genes.mt
heart-genes.mt
muscle-genes.mt
hpo-4.mt
hpo-5.mt
endurance.mt
power.mt
stren.mt
soft.mt


In [5]:
#mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher.mt')

In [6]:
#correct this mt again (more conservative flag):

mt = mt.annotate_rows(flag = hl.if_else(((mt.af_controls > 0.1) | (mt.af_sport > 0.1)), hl.if_else(
    hl.is_defined(mt.gnomad_v3.v3_nfe.AF), hl.if_else(
        ((hl.abs(mt.af_controls - mt.gnomad_v3.v3_nfe.AF) > 0.15) | (hl.abs(mt.af_sport - mt.gnomad_v3.v3_nfe.AF) > 0.15)), 'not', 'ok'), 'not'), 'ok'))

In [7]:
mt = mt.filter_rows(mt.flag == 'ok')

In [8]:
mt.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag.mt')

2021-10-14 18:04:35 Hail: INFO: wrote matrix table with 2108100 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag.mt
    Total size: 2.13 GiB
    * Rows/entries: 2.13 GiB
    * Columns: 1.30 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  4008 rows (5.97 MiB)


In [18]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag.mt')

In [23]:
t = mt.rows()

In [104]:
#t.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag-rows.mt')
t = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag-rows.mt')

colloect p_value tresholds:

In [105]:
brain_vs_controls = t.order_by(t.fisher_brain_vs_controls.p_value).fisher_brain_vs_controls.p_value.take(5)[4]
brain_e_vs_s = t.order_by(t.fisher_brain_e_vs_s.p_value).fisher_brain_e_vs_s.p_value.take(5)[4]
heart_vs_controls = t.order_by(t.fisher_heart_vs_controls.p_value).fisher_heart_vs_controls.p_value.take(5)[4]
heart_e_vs_s = t.order_by(t.fisher_heart_e_vs_s.p_value).fisher_heart_e_vs_s.p_value.take(5)[4]
muscle_vs_controls = t.order_by(t.fisher_muscle_vs_controls.p_value).fisher_muscle_vs_controls.p_value.take(5)[4]
muscle_e_vs_s = t.order_by(t.fisher_muscle_e_vs_s.p_value).fisher_muscle_e_vs_s.p_value.take(5)[4]
hpo4_vs_controls = t.order_by(t.fisher_hpo4_vs_controls.p_value).fisher_hpo4_vs_controls.p_value.take(5)[4]
hpo4_e_vs_s = t.order_by(t.fisher_hpo4_e_vs_s.p_value).fisher_hpo4_e_vs_s.p_value.take(5)[4]
hpo5_vs_controls = t.order_by(t.fisher_hpo5_vs_controls.p_value).fisher_hpo5_vs_controls.p_value.take(5)[4]
hpo5_e_vs_s = t.order_by(t.fisher_hpo5_e_vs_s.p_value).fisher_hpo5_e_vs_s.p_value.take(5)[4]
end_vs_controls = t.order_by(t.fisher_end_vs_controls.p_value).fisher_end_vs_controls.p_value.take(5)[4]
end_e_vs_s = t.order_by(t.fisher_end_e_vs_s.p_value).fisher_end_e_vs_s.p_value.take(5)[4]
power_vs_controls = t.order_by(t.fisher_power_vs_controls.p_value).fisher_power_vs_controls.p_value.take(5)[4]
power_e_vs_s = t.order_by(t.fisher_power_e_vs_s.p_value).fisher_power_e_vs_s.p_value.take(5)[4]
stren_vs_controls = t.order_by(t.fisher_stren_vs_controls.p_value).fisher_stren_vs_controls.p_value.take(5)[4]
stren_e_vs_s = t.order_by(t.fisher_stren_e_vs_s.p_value).fisher_stren_e_vs_s.p_value.take(5)[4]
soft_vs_controls = t.order_by(t.fisher_soft_vs_controls.p_value).fisher_soft_vs_controls.p_value.take(5)[4]
soft_e_vs_s = t.order_by(t.fisher_soft_e_vs_s.p_value).fisher_soft_e_vs_s.p_value.take(5)[4]


### get top variants from each table:

In [106]:
t = t.filter(
((t.fisher_brain_vs_controls.p_value <= brain_vs_controls) & (t.fisher_brain_vs_controls.p_value > 0)) |
    ((t.fisher_brain_e_vs_s.p_value <= brain_e_vs_s) & (t.fisher_brain_e_vs_s.p_value > 0)) |
    ((t.fisher_heart_vs_controls.p_value <= heart_vs_controls)  & (t.fisher_heart_vs_controls.p_value > 0)) |
    ((t.fisher_heart_e_vs_s.p_value <= heart_e_vs_s) & (t.fisher_heart_e_vs_s.p_value > 0)) |
    ((t.fisher_muscle_vs_controls.p_value <= muscle_vs_controls)  & (t.fisher_muscle_vs_controls.p_value > 0)) |
    ((t.fisher_muscle_e_vs_s.p_value <= muscle_e_vs_s) & (t.fisher_muscle_e_vs_s.p_value > 0)) |
    ((t.fisher_hpo4_vs_controls.p_value <= hpo4_vs_controls)  & (t.fisher_hpo4_vs_controls.p_value > 0)) |
    ((t.fisher_hpo4_e_vs_s.p_value <= hpo4_e_vs_s) & (t.fisher_hpo4_e_vs_s.p_value > 0)) |
    ((t.fisher_hpo5_vs_controls.p_value <= hpo5_vs_controls)  & (t.fisher_hpo5_vs_controls.p_value > 0)) |
    ((t.fisher_hpo5_e_vs_s.p_value <= hpo5_e_vs_s) & (t.fisher_hpo5_e_vs_s.p_value > 0)) |
    ((t.fisher_end_vs_controls.p_value <= end_vs_controls) & (t.fisher_end_vs_controls.p_value > 0)) |
    ((t.fisher_end_e_vs_s.p_value <= end_e_vs_s) & (t.fisher_end_e_vs_s.p_value > 0)) |
    ((t.fisher_power_vs_controls.p_value <= power_vs_controls) & (t.fisher_power_vs_controls.p_value > 0)) |
    ((t.fisher_power_e_vs_s.p_value <= power_e_vs_s) & (t.fisher_power_e_vs_s.p_value > 0)) |
    ((t.fisher_stren_vs_controls.p_value <= stren_vs_controls) & (t.fisher_stren_vs_controls.p_value > 0)) |
    ((t.fisher_stren_e_vs_s.p_value <= stren_e_vs_s) & (t.fisher_stren_e_vs_s.p_value > 0)) |
    ((t.fisher_soft_vs_controls.p_value <= soft_vs_controls) & (t.fisher_soft_vs_controls.p_value > 0)) |
    ((t.fisher_soft_e_vs_s.p_value <= soft_e_vs_s) & (t.fisher_soft_e_vs_s.p_value > 0)))

In [107]:
t = t.select(rsid = t.gnomad_v3.rsid,
                                 gnomAD_v3_1_allele_frequency = t.gnomad_v3.v3_nfe.AF,
                                 gnomAD_v3_1_homozygote_count = t.gnomad_v3.v3_nfe.homozygote_count,
                                 predicted_most_severe_consequence = t.gnomad_v3.most_severe_consequence,
                                 gene = t.within_gene,
                                 hpo = t.hpo,
                                 controls_non_ref = t.controls_non_ref,
                                 sports_non_ref = t.sports_non_ref,
                                 controls_ref = t.controls_ref,
                                 sports_ref = t.sports_ref,
                                 speed_non_ref = t.speed_non_ref,
                                 speed_ref = t.speed_ref,
                                 endurance_non_ref = t.endurance_non_ref,
                                 endurance_ref = t.endurance_ref,
                                 fisher_brain_vs_controls = t.fisher_brain_vs_controls.p_value,
                                 fisher_brain_e_vs_s = t.fisher_brain_e_vs_s.p_value,
                                 fisher_muscle_vs_controls = t.fisher_muscle_vs_controls.p_value,
                                 fisher_muscle_e_vs_s = t.fisher_muscle_e_vs_s.p_value,
                                 fisher_heart_vs_controls = t.fisher_heart_vs_controls,
                                 fisher_heart_e_vs_s = t.fisher_heart_e_vs_s.p_value,
                                 fisher_hpo4_vs_controls = t.fisher_hpo4_vs_controls.p_value,
                                 fisher_hpo4_e_vs_s = t.fisher_hpo4_e_vs_s.p_value,
                                 fisher_hpo5_vs_controls = t.fisher_hpo5_vs_controls.p_value,
                                 fisher_hpo5_e_vs_s = t.fisher_hpo5_e_vs_s.p_value,
                                 fisher_end_vs_controls = t.fisher_end_vs_controls.p_value,
                                 fisher_end_e_vs_s = t.fisher_end_e_vs_s.p_value,
                                 fisher_power_vs_controls = t.fisher_power_vs_controls.p_value,
                                 fisher_power_e_vs_s = t.fisher_power_e_vs_s.p_value,
                                 fisher_stren_vs_controls = t.fisher_stren_vs_controls.p_value,
                                 fisher_stren_e_vs_s = t.fisher_stren_e_vs_s.p_value,
                                 fisher_soft_vs_controls = t.fisher_soft_vs_controls.p_value,
                                 fisher_soft_e_vs_s = t.fisher_soft_e_vs_s.p_value,
                                 variant_qc = t.variant_qc)

In [102]:
#t.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-export-fisher-single.ht')

t = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/to-export-fisher-single.ht')

In [108]:
to_export = t.to_pandas()

In [109]:
to_export.to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/fisher-single.csv')

In [110]:
to_export.to_csv('table1.csv')

# rare variants:

In [124]:
t = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag-rows.mt')

In [125]:
selected_genes = brain_genes + heart_genes + skeletal_muscle_genes + Hp_0011804 + Hp_0011805 + endurance_markers_2021 + power_markers_2021 + strength_markers_2021 + soft_tissue_injury_2021

In [127]:
t = t.filter(t.gnomad_v3.v3_nfe.AF < 0.00002)
t = t.filter(hl.any(lambda x: hl.literal(selected_genes).contains(x), t.within_gene))
t = t.filter(t.gnomad_v3.most_severe_consequence == 'stop_gained')

In [128]:
t.gnomad_v3.v3_nfe.AN.show()

,,
locus,alleles,
locus<GRCh38>,array<str>,int32
chr2:15468518,"[""G"",""A""]",64576
chr2:98121411,"[""G"",""T""]",64580
chr2:98162934,"[""G"",""A""]",64586
chr2:178630250,"[""G"",""A""]",64520
chr2:232521386,"[""C"",""T""]",64576
chr2:232541443,"[""G"",""A""]",64564
chr3:155084234,"[""C"",""T""]",64570
chr3:185193077,"[""T"",""A""]",64578


In [115]:
t = t.select(rsid = t.gnomad_v3.rsid,
                                 gnomAD_v3_1_allele_frequency = t.gnomad_v3.v3_nfe.AF,
                                 gnomAD_v3_1_homozygote_count = t.gnomad_v3.v3_nfe.homozygote_count,
                                 predicted_most_severe_consequence = t.gnomad_v3.most_severe_consequence,
                                 gene = t.within_gene,
                                 hpo = t.hpo,
                                 controls_non_ref = t.controls_non_ref,
                                 sports_non_ref = t.sports_non_ref,
                                 controls_ref = t.controls_ref,
                                 sports_ref = t.sports_ref,
                                 speed_non_ref = t.speed_non_ref,
                                 speed_ref = t.speed_ref,
                                 endurance_non_ref = t.endurance_non_ref,
                                 endurance_ref = t.endurance_ref,
                                 variant_qc = t.variant_qc)

In [116]:
to_export = t.to_pandas()

In [121]:
to_export

,locus.contig,locus.position,alleles,rsid,gnomAD_v3_1_allele_frequency,gnomAD_v3_1_homozygote_count,predicted_most_severe_consequence,gene,hpo,controls_non_ref,...,variant_qc.AN,variant_qc.homozygote_count,variant_qc.call_rate,variant_qc.n_called,variant_qc.n_not_called,variant_qc.n_filtered,variant_qc.n_het,variant_qc.n_non_ref,variant_qc.het_freq_hwe,variant_qc.p_value_hwe
0,chr2,15468518,"[G, A]",rs767280377,0.000015,0,stop_gained,[NBAS],"[[HP:0000486:Strabismus, HP:0007663:Reduced vi...",0,...,242,"[120, 0]",0.945312,121,7,0,1,1,0.008264,0.5
1,chr2,98121411,"[G, T]",rs139296152,0.000015,0,stop_gained,[VWA3B],"[[HP:0002061:Lower limb spasticity, HP:0000007...",1,...,242,"[120, 0]",0.945312,121,7,0,1,1,0.008264,0.5
2,chr2,98162934,"[G, A]",rs149236888,0.000015,0,stop_gained,[VWA3B],"[[HP:0002061:Lower limb spasticity, HP:0000007...",1,...,242,"[120, 0]",0.945312,121,7,0,1,1,0.008264,0.5
3,chr2,178630250,"[G, A]",rs140743001,0.000015,0,stop_gained,"[TTN, TTN-AS1]",[[HP:0003687:Centrally nucleated skeletal musc...,0,...,242,"[121, 0]",0.945312,121,7,0,0,0,0.000000,0.5
4,chr2,232521386,"[C, T]",None,0.000000,0,stop_gained,[PRSS56],"[[HP:0008052:Retinal fold, HP:0007906:Ocular h...",0,...,242,"[120, 0]",0.945312,121,7,0,1,1,0.008264,0.5
5,chr2,232541443,"[G, A]",None,0.000015,0,stop_gained,[CHRNG],"[[HP:0000046:Scrotal hypoplasia, HP:0000369:Lo...",0,...,242,"[121, 0]",0.945312,121,7,0,0,0,0.000000,0.5
6,chr3,155084234,"[C, T]",rs150836510,0.000000,0,stop_gained,[MME],"[[HP:0007141:Sensorimotor neuropathy, HP:00090...",0,...,242,"[121, 0]",0.945312,121,7,0,0,0,0.000000,0.5
7,chr3,185193077,"[T, A]",None,0.000000,0,stop_gained,[EHHADH],"[[HP:0002979:Bowing of the legs, HP:0003109:Hy...",0,...,242,"[121, 0]",0.945312,121,7,0,0,0,0.000000,0.5
8,chr4,119158118,"[C, T]",rs374655743,0.000015,0,stop_gained,[MYOZ2],"[[HP:0000006:Autosomal dominant inheritance, H...",0,...,242,"[121, 0]",0.945312,121,7,0,0,0,0.000000,0.5
9,chr5,140842331,"[T, A]",rs782294997,0.000000,0,stop_gained,"[PCDHA1, PCDHA2, PCDHA3, PCDHA4, PCDHA5, PCDHA...",[None],0,...,242,"[121, 0]",0.945312,121,7,0,0,0,0.000000,0.5


In [117]:
to_export.to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/rare.csv')

In [118]:
to_export.to_csv('table2.csv')

# variant burden:

In [ ]:
#mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag.mt')

#mt = mt.filter_rows(mt.gnomad_v3.v3_nfe_AF < 0.05)
#mt = mt.annotate_rows(burden = -hl.log10(mt.gnomad_v3.v3_nfe_AF))
#mt = mt.explode(mt.within_gene)

## overall burden for all variants MAF < 0.05

In [193]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag.mt')
mt = mt.filter_rows(mt.gnomad_v3.v3_nfe.AF < 0.05)
mt = mt.annotate_rows(burden = hl.if_else((mt.gnomad_v3.v3_nfe.AF == 0), 10, -hl.log10(mt.gnomad_v3.v3_nfe.AF)))
mt = mt.annotate_entries(burden_entry = hl.if_else(mt.GT.is_non_ref(), hl.if_else((mt.gnomad_v3.v3_nfe.AF == 0), 10, -hl.log10(mt.gnomad_v3.v3_nfe.AF)), 0))

In [194]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for-burden.mt')

2021-10-14 22:28:15 Hail: INFO: wrote matrix table with 1592569 rows and 126 columns in 3061 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for-burden.mt
    Total size: 1.36 GiB
    * Rows/entries: 1.36 GiB
    * Columns: 1.07 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3030 rows (3.93 MiB)


In [149]:
mt = mt.annotate_cols(burden_all = hl.agg.sum(mt.burden_entry))

In [154]:
columns = mt.cols()

2021-10-14 21:44:31 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [155]:
columns.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag-cols.mt')

2021-10-14 21:47:11 Hail: INFO: Coerced sorted dataset
2021-10-14 21:47:11 Hail: INFO: wrote table with 126 rows in 16 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag-cols.mt
    Total size: 3.98 KiB
    * Rows: 3.97 KiB
    * Globals: 11.00 B
    * Smallest partition: 7 rows (215.00 B)
    * Largest partition:  8 rows (300.00 B)


In [183]:
columns = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/all-fisher-flag-cols.mt')

In [158]:
columns.aggregate(hl.agg.linreg(columns.burden_all, [1,columns.group == 'sport']))  

Struct(beta=[59194.65788302628, 1197.7330484993863], standard_error=[466.3526568842551, 520.8816295613049], t_stat=[126.93110462479451, 2.2994342294393006], p_value=[3.9742592603680084e-133, 0.0231526986582333], multiple_standard_error=2331.7632844212762, multiple_r_squared=0.04089646683657522, adjusted_r_squared=0.03316176092396694, f_stat=5.287397775513385, multiple_p_value=0.023152698658280728, n=126)

In [176]:
columns.aggregate(hl.agg.linreg(columns.burden_all, [1,columns.phenotypes.type == 'speed']))  

Struct(beta=[60445.715182714375, 76.37786309508783], standard_error=[239.27432450294256, 324.56748505367045], t_stat=[252.62098350201808, 0.23532197959526968], p_value=[4.121306075808481e-130, 0.8144934576352295], multiple_standard_error=1550.674852890269, multiple_r_squared=0.0006149153582807321, adjusted_r_squared=-0.010489363359960757, f_stat=0.05537643406506189, multiple_p_value=0.8144934576608334, n=92)

## Burden per gene list:

In [7]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for-burden.mt')

In [4]:
gene_lists = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/gene-lists/sportsmen_gene_lists_october_2021.csv', quote="\"")

2021-10-15 08:35:40 Hail: INFO: Reading table without type imputation
  Loading field 'brain' as type str (not specified)
  Loading field 'sekletal_muscle' as type str (not specified)
  Loading field 'heart' as type str (not specified)
  Loading field 'Hp_0011804' as type str (not specified)
  Loading field 'Hp_0011805' as type str (not specified)
  Loading field 'endurance_markers_2021' as type str (not specified)
  Loading field 'power_markers_2021' as type str (not specified)
  Loading field 'strength_markers_2021' as type str (not specified)
  Loading field 'soft_tissue_injury_2021' as type str (not specified)


In [6]:
brain_genes = gene_lists.brain.collect()
heart_genes = gene_lists.heart.collect()
skeletal_muscle_genes = gene_lists.sekletal_muscle.collect()
Hp_0011804 = gene_lists.Hp_0011804.collect()
Hp_0011805 = gene_lists.Hp_0011805.collect()
endurance_markers_2021 = gene_lists.endurance_markers_2021.collect()
power_markers_2021 = gene_lists.power_markers_2021.collect()
strength_markers_2021 = gene_lists.strength_markers_2021.collect()
soft_tissue_injury_2021 = gene_lists.soft_tissue_injury_2021.collect()

In [8]:
mt.count()

(1592569, 126)

In [9]:
gene_lists = [brain_genes, heart_genes, skeletal_muscle_genes, Hp_0011804, Hp_0011805, endurance_markers_2021, power_markers_2021, strength_markers_2021, soft_tissue_injury_2021]

In [ ]:
for g in gene_lists:
    to_test = mt.filter_rows(hl.any(lambda x: hl.literal(g).contains(x), mt.within_gene))
    to_test = to_test.annotate_cols(burden_all = hl.agg.sum(to_test.burden_entry))
    columns = to_test.cols()
    print(columns.aggregate(hl.agg.linreg(columns.burden_all, [1,columns.group == 'sport'])))
    print(columns.aggregate(hl.agg.group_by(columns.group, hl.agg.stats(columns.burden_all))))
    
    columns = columns.filter(hl.is_defined(columns.phenotypes.type))
    print(columns.aggregate(hl.agg.linreg(columns.burden_all, [1,columns.phenotypes.type == 'speed'])))
    print(columns.aggregate(hl.agg.group_by(columns.phenotypes.type, hl.agg.stats(columns.burden_all))))

2021-10-15 08:37:21 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


Struct(beta=[2981.135811703086, 117.93034553630191], standard_error=[51.02044045559926, 56.98608075485653], t_stat=[58.4302249271531, 2.0694587866748764], p_value=[4.270125449912073e-92, 0.040579994377852785], multiple_standard_error=255.10220227799633, multiple_r_squared=0.03338455626636583, adjusted_r_squared=0.0255892704298043, f_stat=4.282659669743609, multiple_p_value=0.040579994377904466, n=126)
{'control': Struct(mean=2981.1358117030877, stdev=282.4453921758052, min=2092.542799536519, max=3499.4277771366146, n=25, sum=74528.39529257719), 'sport': Struct(mean=3099.066157239388, stdev=245.25556640758518, min=2644.4887880028073, max=4349.301705801201, n=101, sum=313005.68188117817)}
Struct(beta=[3060.656337766805, 86.4346074480148], standard_error=[38.10359376716207, 51.686229294369674], t_stat=[80.32461075638747, 1.6722947026323385], p_value=[1.4466537772010704e-85, 0.097939390214774], multiple_standard_error=246.93951088244728, multiple_r_squared=0.03013656200156456, adjusted_r_s

## Burden per gene:

In [ ]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for-burden.mt')

In [ ]:
gene_mt = mt.group_rows_by(mt.gene).aggregate(
    score = hl.agg.sum(mt.burden_entry)

In [ ]:
gene_mt.describe()

In [ ]:
gene_mt = hl.logistic_regression(y=gene_mt.group, x=gene_mt.score, covariates=[1], test='wald')
gene_mt.logreg.show()

In [ ]:
gene_mt.describe()

In [ ]:
gene_mt.order_by(gene_mt.logreg.p_value).show(n_rows=10)

In [ ]:
#export top 10 genes

treshold = gene_mt.order_by(gene_mt.logreg.p_value).logreg.p_value.take(10)[9]
all_genes = gene_mt.filter(gene_mt.logreg.p_value)

In [ ]:
#now export top 10 results from all lists:

for g in gene_lists:
    to_test = mt.filter_rows(hl.any(lambda x: hl.literal(g).contains(x), mt.within_gene))
    to_test = to_test.annotate_cols(burden_all = hl.agg.sum(to_test.burden_entry))
    columns = to_test.cols()
    print(columns.aggregate(hl.agg.linreg(columns.burden_all, [1,columns.group == 'sport'])))
    print(columns.aggregate(hl.agg.group_by(columns.group, hl.agg.stats(columns.burden_all))))
    
    columns = columns.filter(hl.is_defined(columns.phenotypes.type))
    print(columns.aggregate(hl.agg.linreg(columns.burden_all, [1,columns.phenotypes.type == 'speed'])))



brain_vs_controls = t.order_by(t.fisher_brain_vs_controls.p_value).fisher_brain_vs_controls.p_value.take(5)[4]
brain_e_vs_s = t.order_by(t.fisher_brain_e_vs_s.p_value).fisher_brain_e_vs_s.p_value.take(5)[4]
heart_vs_controls = t.order_by(t.fisher_heart_vs_controls.p_value).fisher_heart_vs_controls.p_value.take(5)[4]
heart_e_vs_s = t.order_by(t.fisher_heart_e_vs_s.p_value).fisher_heart_e_vs_s.p_value.take(5)[4]
muscle_vs_controls = t.order_by(t.fisher_muscle_vs_controls.p_value).fisher_muscle_vs_controls.p_value.take(5)[4]
muscle_e_vs_s = t.order_by(t.fisher_muscle_e_vs_s.p_value).fisher_muscle_e_vs_s.p_value.take(5)[4]
hpo4_vs_controls = t.order_by(t.fisher_hpo4_vs_controls.p_value).fisher_hpo4_vs_controls.p_value.take(5)[4]
hpo4_e_vs_s = t.order_by(t.fisher_hpo4_e_vs_s.p_value).fisher_hpo4_e_vs_s.p_value.take(5)[4]
hpo5_vs_controls = t.order_by(t.fisher_hpo5_vs_controls.p_value).fisher_hpo5_vs_controls.p_value.take(5)[4]
hpo5_e_vs_s = t.order_by(t.fisher_hpo5_e_vs_s.p_value).fisher_hpo5_e_vs_s.p_value.take(5)[4]
end_vs_controls = t.order_by(t.fisher_end_vs_controls.p_value).fisher_end_vs_controls.p_value.take(5)[4]
end_e_vs_s = t.order_by(t.fisher_end_e_vs_s.p_value).fisher_end_e_vs_s.p_value.take(5)[4]
power_vs_controls = t.order_by(t.fisher_power_vs_controls.p_value).fisher_power_vs_controls.p_value.take(5)[4]
power_e_vs_s = t.order_by(t.fisher_power_e_vs_s.p_value).fisher_power_e_vs_s.p_value.take(5)[4]
stren_vs_controls = t.order_by(t.fisher_stren_vs_controls.p_value).fisher_stren_vs_controls.p_value.take(5)[4]
stren_e_vs_s = t.order_by(t.fisher_stren_e_vs_s.p_value).fisher_stren_e_vs_s.p_value.take(5)[4]
soft_vs_controls = t.order_by(t.fisher_soft_vs_controls.p_value).fisher_soft_vs_controls.p_value.take(5)[4]
soft_e_vs_s = t.order_by(t.fisher_soft_e_vs_s.p_value).fisher_soft_e_vs_s.p_value.take(5)[4]